In [2]:
!pip install stable-baselines3
!pip install pyglet
!pip install swig
!pip install box2d-py
!pip install importlib-metadata==4.13.0
!pip install optuna
!pip install sb3_contrib
!pip install easypip

In [3]:
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib import TQC
from stable_baselines3.common.monitor import Monitor
import os
import gym
from stable_baselines3.common.evaluation import evaluate_policy
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances
from typing import Any, Dict
import torch
import torch.nn as nn
from stable_baselines3.common.callbacks import EvalCallback
import torch as th
import numpy as np
from easypip import easyimport
import functools
import time
easyimport("importlib_metadata==4.13.0")
OmegaConf = easyimport("omegaconf").OmegaConf
bbrl_gym = easyimport("bbrl_gym")
bbrl = easyimport("bbrl>=0.1.6")
import os
import copy
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.normal import Normal
from torch.distributions.independent import Independent
import gym
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class
from bbrl.workspace import Workspace
from bbrl.agents import Agents, RemoteAgent, TemporalAgent
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
from bbrl.visu.play import load_agent, play
from bbrl.utils.replay_buffer import ReplayBuffer
from bbrl.utils.functionalb import gae

# !rm -rf gym-rocketlander
!git clone https://github.com/EmbersArc/gym-rocketlander
# !conda uninstall gym-rocketlander
!pip install -e ./gym-rocketlander

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

fatal: destination path 'gym-rocketlander' already exists and is not an empty directory.
Obtaining file:///home/hanzopgp/projects/MasterArtificialIntelligencePTs/S3/RLD/TME_bonus/gym-rocketlander
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gym-rocketlander
    Found existing installation: gym-rocketlander 0.0.1
    Uninstalling gym-rocketlander-0.0.1:
      Successfully uninstalled gym-rocketlander-0.0.1
  Running setup.py develop for gym-rocketlander


In [4]:
import threading

def print_infos(reward):
    threading.Timer(5.0, print_infos).start()
    print(reward)

class RocketLanderWrapper(gym.Wrapper):
    def __init__(self, env):
        super(RocketLanderWrapper, self).__init__(env)
        self.env = env
        self.prev_shaping = None
        
    def reset(self):
        self.prev_shaping = None
        return self.env.reset()

    def step(self, action):
        next_state, reward, done, info = self.env.step(action)
        
        # shaping distance a la cible
        shaping = 0.5 * (1 - self.env.distance) # distance restante a la cible
        
        # shaping pour que le rocket reste droit
        shaping -= 0.5 * (np.abs(self.angle))

        # shaping pour la vitesse du rocket par rapport a sa distance
        shaping -= 0.1 * np.clip((self.speed/(1-self.env.distance)), -0.5, 0.5)
				# shaping -= 0.1 * self.speed

#         shaping = -0.5 * (self.env.distance + self.env.speed + abs(self.env.angle) ** 2)
#         shaping += 0.1 * (self.env.legs[0].ground_contact + self.env.legs[1].ground_contact)
        
#         print(reward, 0.01 * (1 - self.env.distance), 0.03 * (np.abs(self.angle)), 0.003 * (self.speed))
    
        # shaping atterissage
        if (
            self.env.legs[0].ground_contact > 0
            and self.env.legs[1].ground_contact > 0
            and self.env.speed < 0.1
        ):
            print("ATTERISSAGE")
            shaping += 10.0 / self.env.speed
            
        if (
            self.env.legs[0].joint.angle < 0 
            or self.env.legs[1].joint.angle > -0
        ):
            print("JAMBE CASSEE")
#             shaping -= 0.1
            shaping -= 0.5
            
        if (
            self.env.legs[0].ground_contact > 0
            or self.env.legs[1].ground_contact > 0
        ):
            print("GROUND CONTACT")
            shaping += 0.1
            
#         # shaping si on rate la cible
#         if (
#             self.env.lander.position.y <= self.env.ship.position.y # a depasse la position y 
#             and self.env.legs[0].ground_contact == 0 # sans avoir touche la jambe gauche
#             and self.env.legs[1].ground_contact == 0 # ni la jambe droite
#         ):
#             print("CIBLE RATE")
#             shaping -= 20.0 * max(0.1, self.env.speed)
        
#         quand basse vitesse et basse distance : ++  0.1 et 1
#         quand basse vitesse et haute distance : --  0.1 et 10
#         quand haute vitesse et basse distance : --  0.9 et 1
#         quand haute vitesse et longue distance = ++ 0.9 et 10
#         (vitesse*distance)²

        # shaping si le rocket s'explose dans la cible
#         if (
#             ((self.env.legs[0].ground_contact > 0
#             or self.env.legs[1].ground_contact > 0)
#             and self.env.speed >= 0.1)
#         ):
#             print("ROCKET EXPLOSE DANS LA CIBLE")
#             shaping -= 5.0 * max(0.1, self.env.speed)
            
#         if (
#             self.env.lander.position.x < 0 
#             or self.env.lander.position.y < 0 
#         ):
#             print("ROCKET HORS DE LA MAP")
#             shaping -= 1.0
            
#         print("r",reward)
        reward += shaping
#         print("r+s",reward)
#         reward = np.clip(reward, -1, 1)
        return next_state, reward, done, info


class FrameSkip(gym.Wrapper):
    def __init__(self, env: gym.Env, skip: int = 1):
        super().__init__(env)
        self._skip = skip

    def step(self, action: np.ndarray):
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break

        return obs, total_reward, done, info

    def reset(self):
        return self.env.reset()

In [5]:
env = gym.make("RocketLander-v0")
env = RocketLanderWrapper(env)
env = FrameSkip(env, skip=4)
obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

Matplotlib backend: module://matplotlib_inline.backend_inline
The observation space: Box([ -1.  -1.  -1.  -1.  -1.  -1.  -1. -inf -inf -inf], [ 1.  1.  1.  1.  1.  1.  1. inf inf inf], (10,), float32)
The action space: Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)


In [6]:
model = TQC("MlpPolicy", env, verbose=1, seed=2182406907)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
# Relancer a chaque fois jusqu'a ce qu'il y ait plus le temps, il faut visualiser l'atterissage !
model.learn(total_timesteps=100_000) 

JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
JAMBE CASSEE
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 217      |
|    ep_rew_mean     | -4.72    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 42       |
|    time_elapsed    | 20       |
|    total_timesteps | 868

KeyboardInterrupt: 

In [ ]:
testing = gym.make("RocketLander-v0")
# testing = RocketLanderWrapper(testing)
testing = FrameSkip(testing, skip=4)
tst = evaluate_policy(model, testing, return_episode_rewards=True)

In [ ]:
from numpy import mean, std
rewards = tst[0]
print( "min:",min(rewards) , "max:",max(rewards), "mean:", mean(rewards), "std:", std(rewards) )

min: -5.273333422839642 max: -4.940000128000975 mean: -5.08133344501257 std: 0.08228945249266768


In [ ]:
# !apt-get install ffmpeg -y freeglut3-dev -y xvfb -y # For visualization

In [ ]:
# !python --version
# !sudo apt-get install --only-upgrade python3

In [ ]:
# !python --version

In [ ]:
# # Set up fake display; otherwise rendering will fail
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

# import base64
# from pathlib import Path

# from IPython import display as ipythondisplay

# def show_videos(video_path='', prefix=''):
#   """
#   Taken from https://github.com/eleurent/highway-env

#   :param video_path: (str) Path to the folder containing videos
#   :param prefix: (str) Filter the video, showing only the only starting with this prefix
#   """
#   html = []
#   for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
#       video_b64 = base64.b64encode(mp4.read_bytes())
#       html.append('''<video alt="{}" autoplay 
#                     loop controls style="height: 400px;">
#                     <source src="data:video/mp4;base64,{}" type="video/mp4" />
#                 </video>'''.format(mp4, video_b64.decode('ascii')))
#   ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

# def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
#   """
#   :param env_id: (str)
#   :param model: (RL model)
#   :param video_length: (int)
#   :param prefix: (str)
#   :param video_folder: (str)
#   """
#   eval_env = DummyVecEnv([lambda: gym.make(env_id)])
#   # Start the video at step=0 and record 500 steps
#   eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
#                               record_video_trigger=lambda step: step == 0, video_length=video_length,
#                               name_prefix=prefix)

#   obs = eval_env.reset()
#   for _ in range(video_length):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, _, _, _ = eval_env.step(action)

#   # Close the video recorder
#   eval_env.close()

In [ ]:
# record_video('RocketLander-v0', model, video_length=500, prefix='sac-rocket')

In [ ]:
# show_videos('videos', prefix='sac-rocket')

In [ ]:
# model.save("tqc_rocketlander_"+str(mean(rewards)))
model.save("tqc_rocketlander_test14")